# Co-occurrence network null models
with Andy and Miguel

___

In this notebook, we'll explore the use of network null models to identify special structural features of co-occurrence networks, using methods we applied to [Javanese _wayang kulit_](http://jhnr.uni.lu/index.php/jhnr/article/view/42).
___

## Getting started

First, choose a data set you'd like to work with by defining the variable "dataset" in the cell below as one of the following strings:

* 'WayangAdegan' (Javanese _wayang kulit Mahabharata_ , finer scene-level co-occurrence window)
* 'WayangLakon' (Javanese _wayang kulit Mahabharata_ , coarser story-level co-occurrence window)
* 'Mahabharat1988' (B.R. Chopra's 1988 serial TV adaption of the _Mahabharata_ )
* 'Mahabharat2013' (2013 serial TV adaption of the _Mahabharata_ )
* 'AntonyAndCleopatra' (Shakespeare)
* 'RomeoAndJuliet' (Shakespeare)
* 'InfinityWar' (Marvel Comics story arc from comicbookdb.com)
* 'InfinityGauntlet' (Marvel Comics story arc from comicbookdb.com)
* 'InfinityCrusade' (Marvel Comics story arc from comicbookdb.com)
* 'Annihilation' (Marvel Comics story arc from comicbookdb.com)
* 'CivilWar' (Marvel Comics story arc from comicbookdb.com)
* 'InfinityCrusade' (Marvel Comics story arc from comicbookdb.com)
* 'DarkPhoenixSaga' (Marvel Comics story arc from comicbookdb.com)

Alternatively, prepare your own character lists in the same format as the other data files in the folder "./data/", save it there with of the form "YOURTITLE_CharactersByEpisode.txt", and define "dataset = 'YOURTITLE'" in the cell below.

Then run the cells below to load the modules and functions that we will use.

In [ ]:
dataset = 'AntonyAndCleopatra'

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import os
import errno
try:
    os.makedirs("./gephi/")
except OSError as exception:
    if exception.errno != errno.EEXIST:
        raise

In [ ]:
def LoadNetworks(dataset):
    
    EpisodeNodes, CharacterNodes, B = ConstructBipartite(dataset)
    G = CharacterCooccurrenceNetwork(B, EpisodeNodes, CharacterNodes)
    GI = InverseWeightNetwork(G)
    E = EpisodeIntersectionNetwork(B, EpisodeNodes, CharacterNodes)
    EI = InverseWeightNetwork(E)

    return EpisodeNodes, CharacterNodes, B, G, GI, E, EI


def ConstructBipartite(dataset):

    episodelist = open(
        "./data/" + dataset + "_CharactersByEpisode.txt").read().splitlines()

    characters = []
    episodes = dict()
    B = nx.Graph()
    for episode in episodelist:
        episodes[episode.split('=')[0].strip(' ')] = [
        char.strip(" ") for char in episode.split('=')[1].split(',')
        if char.strip(" ")]
        characters.extend([
        char.strip(" ") for char in episode.split('=')[1].split(',')
        if char.strip(" ")])
    EpisodeNodes = sorted(set(episodes.keys()))
    CharacterNodes = sorted(
        set([character.strip(" ") for character in characters]))

    B.add_nodes_from(EpisodeNodes)
    B.add_nodes_from(CharacterNodes)

    for ep in EpisodeNodes:
        for char in episodes[ep]:
            B.add_edge(ep,char)

    return EpisodeNodes, CharacterNodes, B

def CharacterCooccurrenceNetwork(B, EpisodeNodes, CharacterNodes):

    G = nx.Graph()
    G.add_nodes_from(CharacterNodes)
    for c1 in CharacterNodes:
        for c2 in CharacterNodes:
            for L in EpisodeNodes:
                if (B.has_edge(L,c1)) and (B.has_edge(L,c2)) and (c1 is not c2):
                    if G.has_edge(c1,c2):
                        G[c1][c2]['weight'] += 1./2
                    else:
                        G.add_edge(c1,c2, weight=1./2)

    return G

def InverseWeightNetwork(G):

    GI=nx.Graph()
    GI.add_nodes_from(G.nodes())
    for edge in G.edges():
        GI.add_edge(edge[0],edge[1],weight = \
            1./float(G[edge[0]][edge[1]]['weight']))

    return GI

def EpisodeIntersectionNetwork(B, EpisodeNodes, CharacterNodes):

    E = nx.Graph()
    E.add_nodes_from(EpisodeNodes)
    for e1 in EpisodeNodes:
        counter = 0
        for e2 in EpisodeNodes:
            for C in CharacterNodes:
                if (B.has_edge(C,e1)) and (B.has_edge(C,e2)) and (e1 is not e2):
                    counter += 1
                    if E.has_edge(e1,e2):
                        E[e1][e2]['weight'] += 1./2 # because of double-counting
                    else:
                        E.add_edge(e1,e2, weight=1./2)

    return E


___

___

## Part 1: Reviewing "raw" centrality measures

As discussed before, the tools of network analysis provide a means of answering questions like,
_"Which characters play unique structural roles in the network?"_ Our initial motivation in applying network analysis to the Javanese _wayang kulit_ retelling of the _Mahabharata_ was specifically to seek out otherwise-"hidden" patterns within the epic's scaffolding of character encounters that traditional approaches would tend to overlook. However, some aspects of a character's "importance" within a story (such as their overall number of appearances) could be considered, and even quantified, without any need to resort to a network perspective. Our desire to disentangle (insofar far as is possible) a character's structural role from its overall number of appearances motivated our adoption of network null models.

Let's review some network centrality measures on your chosen _character co-occurrence network_ again, and examine how they relate to characters' numbers of appearances in episodes ( i.e. their _degrees_ ).

In [ ]:
EpisodeNodes, CharacterNodes, B, G, GI, E, EI = LoadNetworks(dataset)

CharacterDegree = dict(B.degree(CharacterNodes))
CharacterNodeStrength = nx.degree( G , weight = 'weight' )
CharacterTBetweenness = nx.betweenness_centrality( G )
CharacterBetweenness = nx.betweenness_centrality( GI , weight = 'weight' )
CharacterCloseness = nx.closeness_centrality( G )
CharacterEig = nx.eigenvector_centrality( G , weight = 'weight' )

cdeg = list(CharacterDegree.values())
cstr = [d[1] for d in CharacterNodeStrength]
ctbt = list(CharacterTBetweenness.values())
cbet = list(CharacterBetweenness.values())
cclo = list(CharacterCloseness.values())
ceig = list(CharacterEig.values())

plt.plot(cdeg,cstr,'k.')
plt.xlabel('Node degree')
plt.ylabel('Node strength', fontsize=14)
plt.show()

plt.plot(cdeg,cbet,'k.')
plt.xlabel('Node degree')
plt.ylabel('Betweenness centrality', fontsize=14)
plt.show()

plt.plot(cdeg,ctbt,'k.')
plt.xlabel('Node degree')
plt.ylabel('Betweenness centrality (unweighted)', fontsize=12)
plt.show()

plt.plot(cdeg,cclo,'k.')
plt.xlabel('Node degree')
plt.ylabel('Closeness centrality', fontsize=14)
plt.show()

plt.plot(cdeg,ceig,'k.')
plt.xlabel('Node degree')
plt.ylabel('Eigenvector centrality', fontsize=14)
plt.show()



### Some questions to ponder and discuss:

* What trends do you see? How do each of these centrality measures vary with respect to a character's number of appearances?

* Recalling how these centrality measures are defined, can you explain why you see these general trends? How might our use of weighted networks (instead of networks with unweighted, binary links) affect these trends?

* Our initial goal was to seek out otherwise-_hidden_ structural patterns that we probably wouldn't have noticed without the help of network science. How might the trends observed above this limit our ability to identify overlooked _"characters with unique structural roles"_ using network centrality measures?

* How might we get around this obstacle?

***



___


## Part 2: Identifying centrality outliers using null models

### Refining our question...

Our aim in using network analysis as an exploratory tool was to reveal _otherwise-hidden_ patterns that we probably wouldn't have noticed just by talking about, say, which characters appear most often. Ideally, this approach should bring to the forefront lower-profile, less-familiar characters with interesting roles in the network, as opposed to just being a very roundabout way to recreate lists of the most frequently-appearing characters.

Instead of asking, _"which nodes are important in the network?"_ , maybe we should have been more specific: 
"Which characters have unique structural roles **that can't merely be attributed to how often they appear**?". Let's look for characters who have centrality values that we wouldn't **expect** them to have based on their degrees.


## Null models

In order to do this -- to look for characters having centrality values that we wouldn't **expect** them to have based on their degrees -- we first have to know what we expect. That is, _"What centrality values do we expect a character to have within a co-occurrence network based on its number of appearances?"_ .

We answer this question by generating a bunch of artificial co-occurrence networks in which characters still have the same numbers of appearances, but in which other details are free to vary. Imagine 1000 alternate, parallel universe versions of the story. In each of these parallel universe versions, the frequently-appearing characters appear just as frequently, and one-off characters still appear only once. If some scenes in the real story depict many characters, while others feature only a few, the same will also be true of the story's parallel universe counterparts. However, details of each scene, such as the specific combinations of characters that appear together in each of these scenes, may be completely different. If some feature of the empirical network is shared by all these parallel-universe versions, then their shared features might explain this similarity. However, if the empirical network has some special feature that none of its 1000 parallel-universe counterparts share, then we now have reason to argue that this feature can't be explained just by the thing they have in common (their _degree sequences_). If the empirical network has some special feature that 10 out of its 1000 parallel-universe counterparts share, then we can still perhaps make that argument... just not quite as strongly.

For example, if a character has higher betweenness centrality in the empirical network than it does in 99.9% of null model networks that share the same degree sequence, then we have reason to claim that this character's position "in between" other characters can't just be attributed to its number of appearances, or the degree sequence in general. We can can reject the _null hypothesis_ that its betweenness is explained by its number of appearances, and confidently assert that there must be some more special, not-so-random patterns of co-occurrence going on that place the character in that special position.

So, our goal is to generate a bunch of new networks that share the degree sequence of the empirical network's _degree sequences_ (that is, each character's number of scenes) and underlying _bipartite_ structure. First, let's do it using a _configuration model_.


___



### Configuration model

In a _configuration model_, we take the _degree sequences_ (that is, the list of characters and their numbers of appearances, as well as the list of episodes and their numbers of characters) from our empirical network of character-episode affiliations, and attempt to randomly generate a network that has (approximately) the same degree sequence. We do so buy considering each possible pairing of a character $c$ (with degree $d(c)$ in the empirical network) and episode $e$ (with degree $d(e)$ in the empirical network), and adding a link between them with probability

$$ P \left( c \mathrm{ \ is \ linked \ to \ } e \right) =  \frac{d(c) \times d(e)}{\mathrm{Total \ number \ of \ links}} $$

___
___


#### Configuration model code

The function below generates a network realization according to the configuration model above given the empirical network's degree sequences.

In [ ]:
def BipartiteConfigurationModelGenerate(EpisodeDegrees, CharacterDegrees):

    EpisodeNodes = list(EpisodeDegrees.keys())
    CharacterNodes = list(CharacterDegrees.keys())

    C = nx.Graph()
    C.add_nodes_from(CharacterDegrees.keys())
    C.add_nodes_from(EpisodeDegrees.keys())

    NumberOfLinks = sum(CharacterDegrees.values())

    for character in CharacterNodes:
        for episode in EpisodeNodes: 
            if ( np.random.random() < np.divide(np.multiply(float(CharacterDegrees[character]),
                    float(EpisodeDegrees[episode])), float(NumberOfLinks))):
                C.add_edge(character, episode)

    return C



### More questions to ponder:

1) If a null model network is constructed by generating links randomly in accord with the above probability, will its degree sequence exactly match that of the empirical network?

2) What issues might this _configuration model_ approach introduce in the analysis of low-degree characters?

3) Imagine that you wanted to better understand the role that sizes of episodes played in the features of the co-occurrence network. For example, imagine that in a certain traditional genre of storytelling, it is customary for many scenes to feature only a small number of characters, while a few select scenes, many characters appear at once. Suppose you want to address the research question, _To what extent does this convention affect/explain the features of the resulting co-occurrence network?_ How could you alter the above configuration null model to investigate this?


___

### Generating configuration model networks

Let's now generate an ensemble of _configuration model_ networks to use as a null model for comparison to our story's co-occurrence network. Each time we generate a bipartite network of character-episode affiliations, we will then create the corresponding character co-occurrence network, and then calculate some node and networks metrics on this network and record the results.

After we generate the ensemble by running the cell below, we will then go on to look at some of these results. A "sufficiently large" ensemble of networks is necessary for us to get a good approximation for the probability distributions of each of our network metrics. If later we find that our initial ensemble was too small to produce smooth distributions, we may wish to alter the value of the variable 'EnsembleSize' and then re-run the cell below to improve our results.

In [ ]:
EnsembleSize = 50
Links = False


EpisodeDegree = dict(B.degree(EpisodeNodes))

EnsembleBC = {}
RunningMeanBC = {}
EnsembleCC = {}
RunningMeanCC = {}
EnsembleEC = {}
RunningMeanEC = {}
EnsembleDegree = {}

for character in CharacterNodes:
    EnsembleBC[character] = []
    RunningMeanBC[character] = []
    EnsembleCC[character] = []
    RunningMeanCC[character] = []
    EnsembleEC[character] = []
    RunningMeanEC[character] = []
    EnsembleDegree[character] = []
    
EnsembleDiameter = []
EnsembleShortestPath = []
EnsembleDensity = []
EnsembleClustering = []

if Links:
    EnsembleLinkWeight = {}
    EnsembleLinkBC = {}
    for edge in G.edges():
        EnsembleLinkWeight[tuple(sorted(edge))] = []
        EnsembleLinkBC[tuple(sorted(edge))] = []
        
for t in range(EnsembleSize):
    if ((t%10)==0):
        print("Generating null model network realization ",t+1)
    C = BipartiteConfigurationModelGenerate(EpisodeDegree, CharacterDegree)
    H = CharacterCooccurrenceNetwork(C, EpisodeNodes, CharacterNodes)
    HI = InverseWeightNetwork(H)
    CBetweenness = nx.betweenness_centrality(HI, weight='weight')
    CCloseness = nx.closeness_centrality(H)
    CEigenvector = nx.eigenvector_centrality(H, weight='weight')
    CDegree = nx.degree(C)        
    for character in CharacterNodes:
        EnsembleBC[character].append(CBetweenness[character])
        RunningMeanBC[character].append(np.mean(EnsembleBC[character]))
        EnsembleCC[character].append(CCloseness[character])
        RunningMeanCC[character].append(np.mean(EnsembleCC[character]))
        EnsembleEC[character].append(CEigenvector[character])
        RunningMeanEC[character].append(np.mean(EnsembleEC[character]))
        EnsembleDegree[character].append(CDegree[character])
    if (nx.number_connected_components(H)>1):
        Hcomps = list(nx.connected_component_subgraphs(H))
        HLC = Hcomps[[len(h.nodes()) for h in Hcomps].index(np.max([len(h.nodes()) for h in Hcomps]))]
        HLCI = InverseWeightNetwork(HLC)
        EnsembleDiameter.append(nx.diameter(HLC))
        EnsembleShortestPath.append(nx.average_shortest_path_length(HLCI, weight="weight"))
        EnsembleDensity.append(nx.density(HLC))
        EnsembleClustering.append(nx.average_clustering(HLC,weight="weight"))
    else:
        EnsembleDiameter.append(nx.diameter(HI))
        EnsembleShortestPath.append(nx.average_shortest_path_length(HI, weight="weight"))
        EnsembleDensity.append(nx.density(H))
        EnsembleClustering.append(nx.average_clustering(H,weight="weight"))
    if Links:
        CLinkBetweenness = nx.edge_betweenness_centrality(HI, weight="weight", normalized=True)
        for edge in CLinkBetweenness.keys():
            if tuple(sorted(edge)) in EnsembleLinkWeight.keys():
                EnsembleLinkWeight[tuple(sorted(edge))].append(H[edge[0]][edge[1]]["weight"])
                EnsembleLinkBC[tuple(sorted(edge))].append(CLinkBetweenness[edge])

### Examining null model results

#### Betweenness centrality

Having generated the ensemble of networks, let's see some of its results. 

The cell just below will print a list of characters sorted by their betweenness centrality in descending order for reference.  In the subsequent cell, we can choose a character name from this list by defining the variable "character" as desired (for example, "character = 'Romeo'".). Running the cell will then display the following:

* The values of that character's betweenness centrality in the sequence of null model networks, compared to its empirical network value
* The running mean of the character's null model betweenness centrality values in this sequence. We expect this curve to smooth out and approach to fairly constant value; if it hasn't yet, we may wish to go back a step, increase the value of 'EnsembleSize', and re-run the cell above to generate a larger ensemble.
* A histogram of the character's null model betweenness centrality, with the empirical value and the corresponding "$p$-value". This "$p$-value" approximates the fraction of null model networks in which the measure at hand would exceed the actual value observed in the empirical network. That is to say, it represents the probability that this value could be achieved by random chance in co-occurrence networks with similar degree sequences.

The default is set so that results will be shown for the highest-centrality character. Feel free to explore by re-running the cells below for different choices of the string variable "character".


In [ ]:
print([ (index+1,character) for index,character in enumerate(sorted(CharacterNodes, key=lambda x: CharacterBetweenness[x], reverse=True)) ])


In [ ]:
character = sorted(CharacterNodes, key=lambda x: CharacterBetweenness[x], reverse=True)[0]

actualbc = CharacterBetweenness[character]

plt.plot(range(1,EnsembleSize+1),actualbc*np.ones(EnsembleSize),'r--')
plt.plot(range(1,EnsembleSize+1),EnsembleBC[character],'.')
plt.xlim((0,EnsembleSize+1))
plt.xlabel('Network realization')
plt.ylabel('Betweenness centrality')
plt.title('Node: ' + character )
plt.show()

plt.plot(range(1,EnsembleSize+1),actualbc*np.ones(EnsembleSize),'r--')
plt.plot(range(1,EnsembleSize+1),RunningMeanBC[character])
plt.xlim((1,EnsembleSize+1))
plt.xlabel('Number of realizations')
plt.ylabel('Ensemble mean B.C.')
plt.title('Node: ' + character )
plt.show()
    
n, bins, __ = plt.hist(EnsembleBC[character])

NumberOfBins = 1000

binsc = .5*( bins[1:] + bins[:-1] )
bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
bins2c = .5*( bins2[1:] + bins2[:-1] )
n2 = np.interp(bins2c,binsc,n)

diff = [np.abs(x-actualbc) for x in bins2c]
ind = diff.index(np.min(diff))
pval = np.sum(n2[ind:])/np.sum(n2)

plt.plot([actualbc,actualbc],[0,np.max(n)],'r--')
plt.plot(bins2c,n2,'k')
plt.xlim((bins2c[0],bins2c[-1]))
plt.xlabel('Betweenness centrality')
plt.ylabel('Null model abundance')
plt.title('Node: ' + character , fontsize=16 )
plt.show()

CPD = np.cumsum(n2)/np.sum(n2)
plt.plot(bins2c, CPD,'k')
plt.plot([actualbc,actualbc],[0,CPD[ind]],'r--')
plt.plot([0,bins2c[ind]],[CPD[ind],CPD[ind]],'r--')
plt.xlim((bins2c[0],bins2c[-1]))
plt.ylim((0,1))
plt.xlabel('Betweenness centrality')
plt.ylabel('Cumulative probability distribution')
plt.title('Node: ' + character + " ; $p$-value: 1 - " + str(round(1-pval,4)) + " = " + str(round(pval,4)), fontsize=16 )
plt.show()

#### Closeness centrality

Now, let's do the same for closeness centrality.

In [ ]:
print([ (index+1,character) for index,character in enumerate(sorted(CharacterNodes, key=lambda x: CharacterCloseness[x], reverse=True)) ])

In [ ]:
character = sorted(CharacterNodes, key=lambda x: CharacterCloseness[x], reverse=True)[0]

actualcc = CharacterCloseness[character]

plt.plot(range(1,EnsembleSize+1),actualcc*np.ones(EnsembleSize),'r--')
plt.plot(range(1,EnsembleSize+1),EnsembleCC[character],'.')
plt.xlim((0,EnsembleSize+1))
plt.xlabel('Network realization')
plt.ylabel('Closeness centrality')
plt.title('Node: ' + character )
plt.show()

plt.plot(range(1,EnsembleSize+1),actualcc*np.ones(EnsembleSize),'r--')
plt.plot(range(1,EnsembleSize+1),RunningMeanCC[character])
plt.xlim((1,EnsembleSize+1))
plt.xlabel('Number of realizations')
plt.ylabel('Ensemble mean C.C.')
plt.title('Node: ' + character )
plt.show()
    
n, bins, __ = plt.hist(EnsembleCC[character])

NumberOfBins = 1000

binsc = .5*( bins[1:] + bins[:-1] )
bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
bins2c = .5*( bins2[1:] + bins2[:-1] )
n2 = np.interp(bins2c,binsc,n)

diff = [np.abs(x-actualcc) for x in bins2c]
ind = diff.index(np.min(diff))
pval = np.sum(n2[ind:])/np.sum(n2)

plt.plot([actualcc,actualcc],[0,np.max(n)],'r--')
plt.plot(bins2c,n2,'k')
plt.xlim((bins2c[0],bins2c[-1]))
plt.xlabel('Closeness centrality')
plt.ylabel('Null model abundance')
plt.title('Node: ' + character , fontsize=16 )
plt.show()

CPD = np.cumsum(n2)/np.sum(n2)
plt.plot(bins2c, CPD,'k')
plt.plot([actualcc,actualcc],[0,CPD[ind]],'r--')
plt.plot([0,bins2c[ind]],[CPD[ind],CPD[ind]],'r--')
plt.xlim((bins2c[0],bins2c[-1]))
plt.ylim((0,1))
plt.xlabel('Closeness centrality')
plt.ylabel('Cumulative probability distribution')
plt.title('Node: ' + character + " ; $p$-value: 1 - " + str(round(1-pval,4)) + " = " + str(round(pval,4)), fontsize=16 )
plt.show()

#### Eigenvector centrality

Feel free to try the same for eigenvector centrality below.

In [ ]:
print([ (index+1,character) for index,character in enumerate(sorted(CharacterNodes, key=lambda x: CharacterEig[x], reverse=True)) ])

In [ ]:
character = sorted(CharacterNodes, key=lambda x: CharacterEig[x], reverse=True)[0]

actualec = CharacterEig[character]

plt.plot(range(1,EnsembleSize+1),actualec*np.ones(EnsembleSize),'r--')
plt.plot(range(1,EnsembleSize+1),EnsembleEC[character],'.')
plt.xlim((0,EnsembleSize+1))
plt.xlabel('Network realization')
plt.ylabel('Eigenvector centrality')
plt.title('Node: ' + character )
plt.show()

plt.plot(range(1,EnsembleSize+1),actualec*np.ones(EnsembleSize),'r--')
plt.plot(range(1,EnsembleSize+1),RunningMeanEC[character])
plt.xlim((1,EnsembleSize+1))
plt.xlabel('Number of realizations')
plt.ylabel('Ensemble mean E.C.')
plt.title('Node: ' + character )
plt.show()
    
n, bins, __ = plt.hist(EnsembleEC[character])

NumberOfBins = 1000

binsc = .5*( bins[1:] + bins[:-1] )
bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
bins2c = .5*( bins2[1:] + bins2[:-1] )
n2 = np.interp(bins2c,binsc,n)

diff = [np.abs(x-actualec) for x in bins2c]
ind = diff.index(np.min(diff))
pval = np.sum(n2[ind:])/np.sum(n2)

plt.plot([actualec,actualec],[0,np.max(n)],'r--')
plt.plot(bins2c,n2,'k')
plt.xlim((bins2c[0],bins2c[-1]))
plt.xlabel('Eigenvector centrality')
plt.ylabel('Null model abundance')
plt.title('Node: ' + character , fontsize=16 )
plt.show()

CPD = np.cumsum(n2)/np.sum(n2)
plt.plot(bins2c, CPD,'k')
plt.plot([actualec,actualec],[0,CPD[ind]],'r--')
plt.plot([0,bins2c[ind]],[CPD[ind],CPD[ind]],'r--')
plt.xlim((bins2c[0],bins2c[-1]))
plt.ylim((0,1))
plt.xlabel('Eigenvector centrality')
plt.ylabel('Cumulative probability distribution')
plt.title('Node: ' + character + " ; $p$-value: 1 - " + str(round(1-pval,4)) + " = " + str(round(pval,4)), fontsize=16 )
plt.show()

___

### Centrality outliers in context

We have seen how the null model works to quantify how much an individual node's centrality values deviate from its expected values within the context of the given network's degree sequence. Having focused on individual nodes, now let's step back and summarize the results in context of the whole network.

#### Ranking by $p$-values vs. raw centrality

First, we will check whether or not we actually learn anything new from ranking nodes by $p$-value (from low to high) rather than by the raw values of a centrality measure. The code below computes $p$-values for all characters, and plots their adjusted ranks (#1 means lowest $p$-value) versus their original ranks (#1 means highest centrality). It then lists all characters in order of increasing $p$-value.

To explore other node measures, change the assignment of the variable 'Pval' to the dictionary 'PvalB' (betweenness), 'PvalC' (closeness), or 'PvalE' (eigenvector).

Also feel free to make changes to the list variable 'ShowNames' below if you wish to label some of the nodes (for example, "ShowNames = [ 'Romeo' , 'Juliet' ]" will label those characters).

In [ ]:
def NullModelPvalues(RawCentrality,EnsembleCentrality):

    Pval = {}

    for character in CharacterNodes:

        actualc = RawCentrality[character]

        n, bins = np.histogram(EnsembleCentrality[character])

        NumberOfBins = 1000

        binsc = .5*( bins[1:] + bins[:-1] )
        bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
        bins2c = .5*( bins2[1:] + bins2[:-1] )
        n2 = np.interp(bins2c,binsc,n)

        diff = [np.abs(x-actualc) for x in bins2c]
        ind = diff.index(np.min(diff))

        Pval[character] = np.sum(n2[ind:])/np.sum(n2)
    
    return Pval
    
PvalB = NullModelPvalues(CharacterBetweenness, EnsembleBC)
PvalC = NullModelPvalues(CharacterCloseness, EnsembleCC)
PvalE = NullModelPvalues(CharacterCloseness, EnsembleEC)


In [ ]:
Pval = PvalE
RawCentrality = CharacterEig

ShowNames = sorted(CharacterNodes, key=lambda x: RawCentrality[x], reverse=True)[:5]
    
RawRanking = {}
AdjustedRanking = {}

for index, character in enumerate(sorted(CharacterNodes, key=lambda x: RawCentrality[x], reverse=True)):
    RawRanking[character] = index + 1
    
for index, character in enumerate(sorted(CharacterNodes, key=lambda x: Pval[x])):
    AdjustedRanking[character] = index + 1

for character in CharacterNodes:
    plt.plot(RawRanking[character],AdjustedRanking[character],'.')
    if character in ShowNames:
        plt.annotate(character,(RawRanking[character],AdjustedRanking[character]),color='k',fontsize = 10)
plt.xlabel('Raw centrality ranking', fontsize=14)
plt.ylabel('Adjusted centrality ranking', fontsize=14)
plt.show()

print('Adjusted ranking, Raw ranking, Node, Degree, Raw centrality value, P-value')
print('__________________________________________________________________')
for character in sorted(CharacterNodes, key=lambda x: Pval[x]):
    print(AdjustedRanking[character],RawRanking[character],character,CharacterDegree[character],round(RawCentrality[character],4),round(Pval[character],4))
        

#### Centrality vs. degree: Empirical and null model values

The code below will plot empirical centrality values as a function of degree atop the corresponding null model values. Adjust the list variable "ShowNames" to choose which characters will be labelled in these plots.

The cyan-colored points are values for null model network realizations, blue points are the null model mean values, and red points are the empirical values.

In [ ]:
ShowNames = sorted(CharacterNodes, key=lambda x: CharacterBetweenness[x], reverse=True)[:5]


for character in CharacterNodes:
    plt.plot(CharacterDegree[character]*np.ones(len(EnsembleDegree[character])),EnsembleDegree[character],'c.',markersize=1)
    plt.plot(CharacterDegree[character],np.mean(EnsembleDegree[character]),'b.')
    if character in ShowNames:
        plt.annotate(character,(CharacterDegree[character],CharacterDegree[character]),color='k',fontsize = 10)
plt.plot(cdeg,cdeg,'r.')
plt.xlabel('Degree')
plt.ylabel('Degree (null model)', fontsize=14)
plt.show()

for character in CharacterNodes:
    plt.plot(CharacterDegree[character]*np.ones(len(EnsembleBC[character])),EnsembleBC[character],'c.',markersize=1)
    plt.plot(CharacterDegree[character],np.mean(EnsembleBC[character]),'b.')
    if character in ShowNames:
        plt.annotate(character,(CharacterDegree[character],CharacterBetweenness[character]),color='k',fontsize = 10)
plt.plot(cdeg,cbet,'r.')
plt.xlabel('Degree')
plt.ylabel('Betweenness centrality', fontsize=14)
plt.show()

for character in CharacterNodes:
    plt.plot(CharacterDegree[character]*np.ones(len(EnsembleCC[character])),EnsembleCC[character],'c.',markersize=1)
    plt.plot(CharacterDegree[character],np.mean(EnsembleCC[character]),'b.')    
    if character in ShowNames:
        plt.annotate(character,(CharacterDegree[character],CharacterCloseness[character]),color='k',fontsize = 10)
plt.plot(cdeg,cclo,'r.')
plt.xlabel('Degree')
plt.ylabel('Closeness centrality', fontsize=14)
plt.show()

for character in CharacterNodes:
    plt.plot(CharacterDegree[character]*np.ones(len(EnsembleEC[character])),EnsembleEC[character],'c.',markersize=1)
    plt.plot(CharacterDegree[character],np.mean(EnsembleEC[character]),'b.')    
    if character in ShowNames:
        plt.annotate(character,(CharacterDegree[character],CharacterEig[character]),color='k',fontsize = 10)
plt.plot(cdeg,ceig,'r.')
plt.xlabel('Degree')
plt.ylabel('Eigenvector centrality', fontsize=14)
plt.show()


### Questions:

* What (if anything) did you learn from the plot of $p$-value rankings versus "raw" centrality rankings? Can the null model tell us anything that the centrality measures alone don't?

* Did you observe any characters that have low degree, and low raw centrality values, but greatly exceed the null model expectation (that is, had a low $p$-value)? What does this indicate?

* Do characters that share the above description have anything in common?

* What about characters which have high degree and high centrality, and yet also exceed the null model expectation? Does this tell us anything we didn't know before? How does this distinguish them from other high-degree characters whose centrality values conform more closely to the null model expectation?

* What would a very high $p$-value (approaching 1) indicate? Do you observe any outliers with very high $p$-value? Do these characters share anything in common?

___

## Part 3 : Going further

As a next step, here are some challenges to try. Some relevant codes are attached below, but feel free to try your own approach too.

* __Null model analysis of "global" network metrics:__ The null model analyses generated above kept track of the following "global" network measures in dictionaries: 'EnsembleDiameter', 'EnsembleShortestPath', 'EnsembleDensity', 'EnsembleClustering'. Compare the null model values to the empirical co-occurrence network's properties (Example code is included below). Which metrics differ significantly from their null model expectations? Do these deviations make sense? How do these deviations compare with what we would expect from real-world social networks? Are there any other interesting global network metrics available that would be worth checking? If so, try it.

* __Visualizing network outliers:__ Export null model information and use Gephi to create network visualizations that compare raw centrality measures to their null model $p$-values (you may find the code below to be of use). Does the shift of perspective from raw centrality measures to *p*-values reveal any interesting, previously-hidden structural features?

* __Rewiring model:__ Instead of using a configuration model to generate null model networks, try creating a null model ensemble by randomly rewiring links while preserving the empirical network's degree sequence (you may find the codes included below to be useful). What advantages does this offer over a configuration model? What additional challenges does it present?

* __Episode intersection network:__ Here we have projected the bipartite network of character-episode affiliations onto the set of character nodes to yield weighted _character co-occurrence networks_. If we instead project onto the set of episode nodes, we get am _episode intersection network_ where nodes represent episodes and link weights represent the number of characters appearing in both of the linked episodes. Repeat some of the above analyses on this network (which was actually already generated above as a networkx graph "E" when we imported our networks). How do you interpret centrality measures in this network? What kinds of new questions might this network allow us to investigate?

* __Link betweenness centrality:__ Apply null model analysis to _edge betweenness centrality_ in order to identify _edges_ ( _links_ ) with higher-than-expected betweenness centrality values. How might we interpret these outlier links? Export your results and visualize the results in Gephi along with node-based betweenness centrality values. Does this reveal any interesting new structural features of the network?


## Sample codes

#### Sample codes for "rewiring model":

In [ ]:
def RewireBipartite(B, EpisodeDegrees, CharacterDegrees, NumberOfIterations):

    for t in range(NumberOfIterations):
        char1 = np.random.choice(CharacterNodes)
        episode1 = np.random.choice(list(B.neighbors(char1)))
        char2 = np.random.choice(CharacterNodes)
        episode2 = np.random.choice(list(B.neighbors(char2)))
        while (char1==char2) or (episode1==episode2) or \
            B.has_edge(char1,episode2) or B.has_edge(char2,episode1):
            char1 = np.random.choice(CharacterNodes)
            episode1 = np.random.choice(list(B.neighbors(char1)))
            char2 = np.random.choice(CharacterNodes)
            episode2 = np.random.choice(list(B.neighbors(char2)))
        B.remove_edge(char1,episode1)
        B.remove_edge(char2,episode2)
        B.add_edge(char1,episode2)
        B.add_edge(char2,episode1)
        if (nx.number_connected_components(B)>1):
            B.add_edge(char1,episode1)
            B.add_edge(char2,episode2)
            B.remove_edge(char1,episode2)
            B.remove_edge(char2,episode1)

    return B

In [ ]:
EnsembleSize = 30
NumberOfRewirings = 10

EpisodeDegree = dict(B.degree(EpisodeNodes))

EnsembleBC = {}
RunningMeanBC = {}
for character in CharacterNodes:
    EnsembleBC[character] = []
    RunningMeanBC[character] = []

for t in range(EnsembleSize):
    C = RewireBipartite(B, EpisodeDegree, CharacterDegree, NumberOfRewirings)
    G = CharacterCooccurrenceNetwork(C, EpisodeNodes, CharacterNodes)
    GI = InverseWeightNetwork(G)
    CBetweenness = nx.betweenness_centrality(GI, weight='weight')
    for character in CharacterNodes:
        EnsembleBC[character].append(CBetweenness[character])
        RunningMeanBC[character].append(np.mean(EnsembleBC[character]))

#### Sample code for null model analysis of "global" network metrics:

In [ ]:
actual = nx.average_clustering(G)
EnsembleM = EnsembleClustering

n, bins = np.histogram(EnsembleM)

NumberOfBins = 1000

binsc = .5*( bins[1:] + bins[:-1] )
bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
bins2c = .5*( bins2[1:] + bins2[:-1] )
n2 = np.interp(bins2c,binsc,n)

diff = [np.abs(x-actual) for x in bins2c]
ind = diff.index(np.min(diff))

Pval = np.sum(n2[ind:])/np.sum(n2)

n, bins, __ = plt.hist(EnsembleM)

NumberOfBins = 1000

binsc = .5*( bins[1:] + bins[:-1] )
bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
bins2c = .5*( bins2[1:] + bins2[:-1] )
n2 = np.interp(bins2c,binsc,n)

diff = [np.abs(x-actual) for x in bins2c]
ind = diff.index(np.min(diff))
pval = np.sum(n2[ind:])/np.sum(n2)

plt.plot([actual,actual],[0,np.max(n)],'r--')
plt.plot(bins2c,n2,'k')
plt.xlim((bins2c[0],bins2c[-1]))
plt.xlabel('Network density')
plt.ylabel('Null model abundance')
plt.show()

CPD = np.cumsum(n2)/np.sum(n2)
plt.plot(bins2c, CPD,'k')
plt.plot([actualbc,actualbc],[0,CPD[ind]],'r--')
plt.plot([0,bins2c[ind]],[CPD[ind],CPD[ind]],'r--')
plt.xlim((bins2c[0],bins2c[-1]))
plt.ylim((0,1))
plt.xlabel('Network density')
plt.ylabel('Cumulative probability distribution')
plt.title("$p$-value: 1 - " + str(round(1-pval,4)) + " = " + str(round(pval,4)), fontsize=16 )
plt.show()

#### Sample codes for visualization of centrality outliers:

In [ ]:
def CooccurrenceNetworkNodesForGephi(dataset, CharacterNodes, Deg, Str, Bet, Clo, Eig, BetP, CloP, EigP ):

    b = open("./gephi/" + dataset + "_CharactersNodeList.csv" , 'w')
    b.write("Id,Label,Degree," +
        "Node strength,Betweenness centrality,Closeness centrality," +
        "Eigenvector centrality,P-value (B.C.),P-value (B.C.),P-value (E.C.)\n")
    for char in CharacterNodes:
        charp = char.replace(",","")
        b.write(charp + "," + charp + "," + 
            str(Deg[char]) + "," + 
            str(Str[char]) + "," + 
            str(Bet[char]) + "," +
            str(Clo[char]) + "," +
            str(Eig[char]) + "," + 
            str(BetP[char]) + "," +
            str(CloP[char]) + "," +
            str(EigP[char]) + "\n" )
        
    b.close()

def CooccurrenceNetworkEdgesForGephi(datasettag, G, LinkBetweenness=[], PvalLB=[]):

    b = open("./gephi/" + datasettag + "_CharactersEdgeList.csv" , 'w')
    b.write("Source,Target,Weight")
    if LinkBetweenness:
        b.write(",Link betweenness centrality")
    if PvalLB:
        b.write(",P-value (Link B.C.)")
    b.write("\n")

    for edge in sorted(list(G.edges())):
        edge0 = edge[0].replace(",","")
        edge1 = edge[1].replace(",","")
        b.write(edge0 + "," + edge1 + "," + str(G[edge[0]][edge[1]]['weight']) )
        if LinkBetweenness:
            b.write("," + str(LinkBetweenness[edge]))
        if PvalLB:
            b.write("," + str(PvalLB[edge]))
        b.write("\n")
        
    b.close()

CooccurrenceNetworkNodesForGephi(dataset, CharacterNodes, CharacterDegree,
    CharacterNodeStrength, CharacterBetweenness, CharacterCloseness, CharacterEig,
    PvalB, PvalC, PvalE )

CooccurrenceNetworkEdgesForGephi(dataset, G) #, LinkBetweenness, PvalLBC)
    

#### Sample code for link betweenness centrality:

In [ ]:
def NullModelPvaluesLinks(RawCentrality,EnsembleCentrality):

    Pval = {}

    for edge in RawCentrality.keys():

        actual = RawCentrality[edge]

        n, bins = np.histogram(EnsembleCentrality[edge])

        NumberOfBins = 1000

        binsc = .5*( bins[1:] + bins[:-1] )
        bins2 = np.linspace(binsc[0],binsc[-1],NumberOfBins)
        bins2c = .5*( bins2[1:] + bins2[:-1] )
        n2 = np.interp(bins2c,binsc,n)

        diff = [np.abs(x-actual) for x in bins2c]
        ind = diff.index(np.min(diff))

        Pval[edge] = np.sum(n2[ind:])/np.sum(n2)
    
    return Pval

LinkBetweenness = nx.edge_betweenness_centrality( GI , weight="weight", normalized=True )
PvalLBC = NullModelPvaluesLinks(LinkBetweenness,EnsembleLinkBC)
